
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Głębokie sieci warstwowe</big></big></big></big></big>

---

In [3]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

#from bokeh.io import gridplot, output_file, show
#from bokeh.plotting import figure, output_notebook
#from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

# Głębokie sieci warstwowe
1. w jaki sposób nauczyć pojedyncze neurony?
  * bardzo podstawowe modele nieskończenie wymiarowe
    * wykorzystywane przez sieci RBF
    * jesli $\phi(x)$ będzie wystarczająco wysoko wymiarowe, to dopasuje się do zbioru uczącego
    * generalizacja słaba
  * wyliczyć je ręcznie
  * wykorzystać modele __głębokie__ i nauczyć się $\phi()$
    * typowo używamy funckji aktywacji _rectified linear_ __ReLU__$(x)=max(0, x)$


## Funkcje kosztu
1. __maximum likelihood__
$$L(w)=\mathbb{E}_{x,y\sim\widehat{p}_{DATA}}\log\,p_{model}(y\mid x)$$
  * jeśli zakładmy konkretny rozkład prawdopodobieństwa warunkowego, to dostaniemy konkretną funkcję kosztu
  * dla $p_{model}(y\mid x)=\mathcal{N}(y; f(x;w), \mathbf{I})$ dostaniemy 
  $$L(w)=\frac{1}{2}\mathbb{E}_{x,y\sim\widehat{p}_{DATA}}\|y-f(x;w\|^2+const$$

## Neurony wyjściowe
### __liniowe__ $y=w^Th+b$
  * dla średniej warunkowego rozkładu gausowskiego $$p(y\mid x)\mathcal{N}(y; f(x;w), \mathbf{I})$$
    * można też przewidywać macierz kowariancji $\Sigma$
      * jednak musi być dodatnio określona
      * zwykle ogranicza się do $\Sigma=\sigma\mathbb{I}$
      
### __sigmoidalne__ dla rozkładu Bernoulliego $y=\sigma(W^Th+b)$
  * nigdy nie osiąga wartosci granicznych
  * użycie liniowego obiętego do przedziału $[0,1]$ jest trudne w uczeniu
    * po przejściu przez wartości graniczne gradient byłby zerowy

### __softmax__ dla wielowymiarowego rozkładu Bernoulliego
  * dla zmiennych dyskretnych o $n$ możliwych wartościach
    * warstwa liniowa $z=w^Th+b$
    gdzie $z_i=\log\,P(y=i\mid x)$
    * aby znaleźć $\widehat{y}$ $$softmax\,(z)_i=\frac{\exp(z_i)}{\sum_j\exp(z_j)}$$
    * będzie działac prawidłowo przy uczeniu przez maximum log-likelihood
    $$\log\,softmax(z)_i=z_i-\log\,\sum_j\exp(z_j)$$
      * pierwszy składnik będzie pchany do góry, pozostałe w dół
    * drugi składnik można aproksymować 
    $$\log\,\sum_j\exp(z_j)\simeq\max_j\,z_j$$
      * najsilniej będzie karana największa z niepoprawnych odpowiedzi
      * jeśli największą wartość ma już $z_i$, to
      $$\log\,\sum_j\exp(z_j)\simeq\max_j\,z_j=z_i$$
      i oba składniki będą się praktycznie zrowały
        * przykład nie bedzie miał wielkiego wpływu na uczenie
  * może być użyty dla neuronów ukrytych
    * by neuron wybierał jedną z wielu wartości
  * funkcje celu różne od _log-likelihood_ __nie będą__ dobrze współpracowały z _softmax_
    * w szczególności będzie nią błąd kwadratowy
  * ustabilizowana wersja softmax
    * dla dużych wartości $z$, softmax bedzie się wypłaszczał bardzo
    * _softmax_ jest odporny na translację_
    * można uniezależnić od ekstremalnych wartosci przez
    $$softmax(z)=softmax(z-\max_j\,z_j)$$
    
### __gausowskie__
* może reprezentować warunkowy rozkład gausowski dla $y$ pod warunkiem $x$
  * dla ustalonego $\sigma$
  * dla uczonej macierzy kowariancji $\Sigma$
    * rzadko kiedy innej niż diagonalna

## Neurony __ukryte__
* nigdy nie ma całkowitej pewnosci neurony którego typu będą działały najlepiej
* nie wszystkie typy są __wszędzie__ różniczkowalne
  * __ReLU__ $\max(0,x)$ nie jest w $0$
  * nie oczekujemy, że sieć rzeczywiscie dotrze do punktu o gradiencie $0$
    * stąd punkty o nieokreślonym gradiencie są dopuszczalne
    * jeśli pochodne lewa i prawa są różne, to oprogramowanie zwykle zwraca jedną z nich
      * którą? to już kwestia gustu$\ldots$
    * rzeczywista wartość zwykle i tak różni się o $\epsilon$ 

### Rectified Linear Unit __ReLU__
1. aktywacja $$g(z)=\max(0, z)$$
  * pochodna dodatnia kiedykolwiek neuron jest aktywny
  * druga pochodna prawie wszędzie jest zerowa
2. neurony __nie uczą__ się w miejscach, gdzie nie są aktywne
  * niezerowe nachylenie dla $z_i<0$
  $$g(z;\alpha)_i=max(0,z_i)+\alpha_i\min(0, z_i)$$
  * do wykorzystania, gdy cechy są inwariantne na odwrócenie odczytu, np. całkowita zmiana oświetlenie
    * __stałe__ $\alpha_i$ w __Leaky ReLU__
    * __parametryczne__ $\alpha_i$ gdy $\alpha_i$ jest douczane
3. __maxout__ 
  * złożenie wielu wartości liniowych $z_i$
  $$g(z)_i=\max_{j\in{}Ind(i)}z_j$$
  * maxout uczy się odcinkowo liniowej wypukłej funkcji
4. wybór przez założenie o prostocie
  * łatwiej uczyć modele, jesli są bardziej podobne do liniowych

### Neurony ukryte z sigmoidalnymi funkcjami aktywacji
1. __logistyczna__ $g(z)=\sigma(z)$
2. __tanh__ $$g(z)=\tanh(z)=2\sigma(z)-1$$
  * saturuje podobnie jak logistyczna
  * w okolicach zera uczenie przypomina uczenie sieci liniowych
  

### Inne neurony ukryte
1. jest bardzo wiele funkcji aktywacji
2. __funkcja liniowa__
  * zwykle tylko nietóre
3. __softmax__
  * rzadko, ale mogą wymusić pewne wybory
4. __radialne__
5. __softplus__
$$g(z)=\log(1+\exp(z))$$
  * wydaje się wygładzoną wersją ReLU
  * doświadczalnie sprawuje się słabiej
5. __ograniczony $\tanh$__
$$g(z)=\max(-1,\min(1,z))$$